In [146]:
from __future__ import annotations
import typing
from dataclasses import dataclass, field
import warnings
from contextlib import contextmanager
import itertools
import functools

In [147]:
import numpy as np
import matplotlib.pyplot as plt

In [148]:
@contextmanager
def localize_globals(*exceptions: str, restore_values: bool = True):
    exceptions: typing.Set[str] = set(exceptions)

    old_globals: typing.Dict[str, typing.Any] = dict(globals())
    allowed: typing.Set[str] = set(old_globals.keys())
    allowed.update(exceptions)

    yield None

    new_globals: typing.Dict[str, typing.Any] = globals()

    for name in tuple(new_globals.keys()):
        if name not in allowed:
            del new_globals[name]
    
    if not restore_values:
        return
    
    new_globals.update(
        {k: v for k, v in old_globals.items() if k not in exceptions}
    )

In [149]:
# Not actual typevars, but just exist to satisfy the typechecker
N = typing.TypeVar('N')
M = typing.TypeVar('M')

In [150]:
@dataclass
class QRDecomposition:
    Q: np.ndarray['N,N']
    R: np.ndarray['N,N']
    
    @staticmethod
    def decompose(A: np.ndarray['N,N']) -> QRDecomposition:
        A = A.T
        
        es = np.zeros_like(A)
        es[0] = A[0] / np.linalg.norm(A[0])
        
        for i in range(1, len(A)):
            u = A[i].copy()
            for j in range(i):
                u -= np.dot(es[j], A[i]) * es[j]
            es[i] = u / np.linalg.norm(u)
        
        # tmp = np.zeros_like(A)
        # for i in range(len(A)):
        #     for j in range(i, len(A)):
        #         tmp[i, j] = np.dot(es[i], A[j])
        
        return QRDecomposition(es.T, np.triu(es @ A.T))


In [151]:
with localize_globals():
    A = np.array([
        [1, 2, 3],
        [4, 5, 6],
        [7, 8, 0],
    ], dtype=float)
    
    dec = QRDecomposition.decompose(A)
    
    assert np.allclose(dec.Q @ dec.R, A)
    
    for i, j in itertools.product(range(A.shape[0]), range(A.shape[1])):
        assert np.allclose(dec.Q[i] @ dec.Q[j].T, float(i == j))
    
    print(dec)
    print(np.linalg.qr(A))

QRDecomposition(Q=array([[ 0.12309149,  0.90453403, -0.40824829],
       [ 0.49236596,  0.30151134,  0.81649658],
       [ 0.86164044, -0.30151134, -0.40824829]]), R=array([[8.1240384 , 9.6011363 , 3.32347026],
       [0.        , 0.90453403, 4.52267017],
       [0.        , 0.        , 3.67423461]]))
(array([[-0.12309149,  0.90453403,  0.40824829],
       [-0.49236596,  0.30151134, -0.81649658],
       [-0.86164044, -0.30151134,  0.40824829]]), array([[-8.1240384 , -9.6011363 , -3.32347026],
       [ 0.        ,  0.90453403,  4.52267017],
       [ 0.        ,  0.        , -3.67423461]]))


In [152]:
def get_eigenvalues(A: np.ndarray['N,N'], iterations: int = 500) -> tuple[np.ndarray['N'], np.ndarray['N,N']]:
    """
    Returns eigenvalues and eigenvectors of a matrix A.
    """
    
    A = A.copy()
    for _ in range(iterations):
        dec = QRDecomposition.decompose(A)
        A = dec.R @ dec.Q
    return np.diag(A)

In [212]:
def svd(A: np.ndarray['N,M'], iterations: int = 500) -> tuple[np.ndarray['N,N'], np.ndarray['min(N,M)'], np.ndarray['M,M']]:
    """
    Computes the singular value decomposition of matrix `A` via the QR algorithm.
    
    Returns (U, S, V.T) such that A = U @ np.diag(S) @ V.T.
    """
    
    transposed: bool = False
    
    if A.shape[0] > A.shape[1]:
        A = A.T
        transposed = True
    
    Sigma = A @ A.T
    U = np.eye(A.shape[0])
    
    for _ in range(iterations):
        dec = QRDecomposition.decompose(Sigma)
        Sigma = dec.R @ dec.Q
        U = U @ dec.Q
    
    Sigma = np.sqrt(np.diag(Sigma))
    
    tmp = np.zeros_like(A)
    tmp[:Sigma.shape[0], :Sigma.shape[0]] = np.diag(Sigma ** -1)
    
    # A = U @ Sigma @ V
    V = tmp.T @ U.T @ A
    
    if transposed:
        V, U = U, V
    
    return U, Sigma, V


In [214]:
with localize_globals():
    A = np.array([
        [1, 2, 3, 4],
        [5, 6, 7, 8],
        [9, 0, 1, 2],
    ], dtype=float)
    
    my_result = svd(A)
    np_result = np.linalg.svd(A)
    
    print(my_result)
    print(np_result)
    
    Sigma = np.zeros_like(A)
    np.fill_diagonal(Sigma, np_result[1])
    
    assert np.allclose(my_result[0] @ Sigma @ my_result[2], A)


(array([[ 0.32827266, -0.27258864,  0.9043962 ],
       [ 0.84272698, -0.34796774, -0.41076719],
       [ 0.42667117,  0.89700272,  0.1154895 ]]), array([15.35238785,  7.31734162,  0.87218046]), array([[ 0.54597032,  0.37211848,  0.47618508,  0.58025168],
       [ 0.82825122, -0.35982791, -0.32204829, -0.28426868],
       [-0.12615989, -0.7519209 , -0.05353502,  0.64485086],
       [ 0.        ,  0.        ,  0.        ,  0.        ]]))
(array([[-0.32827266, -0.27258864,  0.9043962 ],
       [-0.84272698, -0.34796774, -0.41076719],
       [-0.42667117,  0.89700272,  0.1154895 ]]), array([15.35238785,  7.31734162,  0.87218046]), array([[-5.45970322e-01, -3.72118480e-01, -4.76185080e-01,
        -5.80251681e-01],
       [ 8.28251224e-01, -3.59827908e-01, -3.22048295e-01,
        -2.84268681e-01],
       [-1.26159891e-01, -7.51920905e-01, -5.35350204e-02,
         6.44850864e-01],
       [ 1.31292109e-16,  4.08248290e-01, -8.16496581e-01,
         4.08248290e-01]]))
